### Training with Pretrained Models (BERT and GPT-2) and CNN Layers

In [28]:
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import TFBertModel, TFGPT2Model, BertTokenizer, GPT2Tokenizer
from tensorflow.keras.p`reprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

# Load and preprocess the dataset
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data = train_data
test_data = test_data

max_len = 50  # Adjust as needed
embedding_dim = 50  # Adjust as needed

# Tokenize and pad sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['text'])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['text']), maxlen=max_len, padding='post')
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['text']), maxlen=max_len, padding='post')

# Labels
y_train = train_data['label']
y_test = test_data['label']

# Step 2: Load BERT and GPT-2 models
bert_model = TFBertModel.from_pretrained('bert-base-uncased', trainable=True)
gpt2_model = TFGPT2Model.from_pretrained('gpt2', trainable=True) 

# Step 3: Define the neural net using BERT and GPT-2 models
input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32)
bert_output = bert_model(input_ids)[0]  # Using [1] to get pooled output
gpt2_output = gpt2_model(input_ids)[0]  # GPT-2 output

# Repeat BERT output for each time step in the sequence
concatenated_output = tf.keras.layers.Concatenate(axis=-1)([bert_output, gpt2_output])

# Reshape the concatenated output
reshaped_output = tf.keras.layers.Reshape((max_len, -1))(concatenated_output)

num_filters = 64
filter_size = 3
cnn_layer = tf.keras.layers.Conv1D(filters=num_filters, kernel_size=filter_size, activation='relu')(reshaped_output)
maxpool_layer = tf.keras.layers.GlobalMaxPooling1D()(cnn_layer)

# Additional layers
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(maxpool_layer)

model = tf.keras.Model(inputs=input_ids, outputs=output_layer)

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])  # Change loss function

early_stopping = tf .keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train, epochs=1, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

214/214 [==============================] - 1037s 5s/step - loss: 0.3026 - accuracy: 0.6962 - val_loss: 0.2747 - val_accuracy: 0.7253


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

def evaluate_sentence(sentence, model, tokenizer, max_len):
    # Tokenize and pad the input sentence
    sequence = tokenizer.texts_to_sequences([sentence])
    padded_sequence = pad_sequences(sequence, maxlen=max_len, padding='post')

    # Print for debugging
    print(f'Sequence: {sequence}')
    # print(f'Padded Sequence: {padded_sequence}')

    # Make prediction using the trained model
    likelihood = model.predict(padded_sequence)[0][0]

    return likelihood

# Example usage:
sentence_to_evaluate = "He don’t come here no more."
result = evaluate_sentence(sentence_to_evaluate, model, tokenizer, max_len)
if result<0.6:
    print("Grammatically \033[1m INCORRECT \033[0m with probability: ",1- result)
else:
    print("Grammatically \033[1m CORRECT \033[0m with probability: ",result)


In [83]:
import json  

model.save("fine_tuned_model.h5")  # Save as HDF5

# Save the tokenizer configuration
tokenizer_config = tokenizer.to_json()
with open('tokenizer_config.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_config, ensure_ascii=False))

In [5]:
# Step 7: Evaluate the performance on the provided test set
test_data = pd.read_csv('test.csv')
test_sequences = tokenizer.texts_to_sequences(test_data['text'])
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_len, padding='post')

predictions = model.predict(padded_test_sequences)

17/17 [==============================] - 22s 1s/step


In [11]:
min_value = np.min(predictions)
max_value = np.max(predictions)
scaled_values = (predictions - min_value) / (max_value - min_value)

binary_predictions = np.where(scaled_values > 0.5, 1, 0)
y_true = test_data['label'].astype(int)

# Calculate precision and recall
precision = precision_score(y_true, binary_predictions)
recall = recall_score(y_true, binary_predictions)

print(f'Precision: {precision}, Recall: {recall}')

Precision: 0.6936416184971098, Recall: 0.9863013698630136


In [76]:
# Step 6: Evaluate a sample sentence
scaled_likelihood = mygrader.evaluate("she swims")
print(f'Likelihood that the sentence is grammatically correct: {scaled_likelihood}')

# Step 7: Evaluate another sentence
scaled_likelihood_new = mygrader.evaluate("she am swimming")
print(f'Likelihood that the sentence is grammatically correct: {scaled_likelihood_new}')

1/1 [==============================] - 0s 90ms/step
Likelihood that the sentence is grammatically correct: 0.1065351739525795
1/1 [==============================] - 0s 76ms/step
Likelihood that the sentence is grammatically correct: 0.1336185485124588


In [49]:
# Step 5: Create an instance of Grader
mygrader = Grader(model, tokenizer, max_len)

In [50]:
# Step 6: Evaluate a sample sentence
likelihood = mygrader.evaluate("she swimmed")
print(f'Likelihood that the sentence is grammatically correct: {likelihood}')

1/1 [==============================] - 0s 228ms/step
Likelihood that the sentence is grammatically correct: 0.8868378400802612
